In this Notebook, we will do basic minimum with spark streaming.<br/>We will save tweets to mongodb.
<br/> Spark required java 8

### Before starting this code run 
Change the hashtags in variable WORDS<br/>
run : python tweepy_connection.py

In [5]:
# Give path to the spark directory.
import findspark
findspark.init('/home/ashwani/spark/spark-2.4.3-bin-hadoop2.7')

In [11]:
#Import Modules
#    Spark
from pyspark import SparkConf, SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
#    Kafka
from pyspark.streaming.kafka import KafkaUtils
#    json parsing
import json
import os
from pymongo import MongoClient

In [12]:
#Mongo db Connection
MONGO_HOST= 'mongodb://172.18.0.2:27017/'
client = MongoClient(MONGO_HOST)
db = client.twitterdb
coll = db.general

In [13]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars spark-streaming-kafka-assembly_2.10-1.6.0.jar pyspark-shell' 
#note that the "pyspark-shell" part is very important!!.

In [14]:
# Create a local StreamingContext and batch interval of 10 seconds
conf = SparkConf().setAppName("Kafka Streaming")
sc = SparkContext(conf = conf)
sc.setLogLevel("WARN")

ssc = StreamingContext(sc, 10)


In [15]:
# spark-submit --jars spark-streaming-kafka-assembly_2.10-1.5.2.jar ./spark-kafka.py

In [16]:
# Connect to kafka
# consumer group : spark-streaming
# topic : twitter
# Number of partition: 1
kafkaStream = KafkaUtils.createStream(ssc, 'localhost:2181', 'spark-streaming', {'twitter':5})


________________________________________________________________________________________________

  Spark Streaming's Kafka libraries not found in class path. Try one of the following.

  1. Include the Kafka library and its dependencies with in the
     spark-submit command as

     $ bin/spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8:2.4.3 ...

  2. Download the JAR of the artifact from Maven Central http://search.maven.org/,
     Group Id = org.apache.spark, Artifact Id = spark-streaming-kafka-0-8-assembly, Version = 2.4.3.
     Then, include the jar in the spark-submit command as

     $ bin/spark-submit --jars <spark-streaming-kafka-0-8-assembly.jar> ...

________________________________________________________________________________________________




TypeError: 'JavaPackage' object is not callable

In [ ]:
# Parse the inbound message as json
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [ ]:
# Number of tweets in the batch
parsed.count().map(lambda x:'Tweets in this batch: %s' % x).pprint()

In [ ]:
# Extract Author name from each tweet
authors_dstream = parsed.map(lambda tweet: tweet['user']['screen_name'])

In [ ]:
# Count the number of tweets per author
author_counts = authors_dstream.countByValue()
author_counts.pprint()

In [ ]:
# Sort the author count
author_counts_sorted_dstream = author_counts.transform(\
  (lambda foo:foo\
   .sortBy(lambda x:( -x[1]))))


In [ ]:
# print top 5 authors
# DStream does not have take attribute. so we use transporm to read each rdd.
top_five_authors = author_counts_sorted_dstream.transform\
  (lambda rdd:sc.parallelize(rdd.take(5)))
top_five_authors.pprint()

In [ ]:
# Get authors with more than one tweet, or whose username starts with 'rm'
filtered_authors = author_counts.filter(lambda x:\
                                                x[1]>1 \
                                                or \
                                                x[0].lower().startswith('rm'))
filtered_authors.transform\
  (lambda rdd:rdd\
  .sortBy(lambda x:-x[1]))\
  .pprint()

In [ ]:
# List the most common words in the tweets
parsed.\
    flatMap(lambda tweet:tweet['text'].split(" "))\
    .countByValue()\
    .transform\
      (lambda rdd:rdd.sortBy(lambda x:-x[1]))\
    .pprint()

In [ ]:
print(sc.uiWebUrl)

## Dstream - sparkStream ##
Dstream is a collection of RDD saved on cache as a batch process.<br/>Each RDD will be a list of twitter data. the list can be of any length.<br/>The object type of element in list is 'str'. We use json.loads to convert it to dict object.

In [ ]:
def f(rdd):
    for t in rdd.collect():
        coll.insert_one(json.loads(t))

In [ ]:
sparkStream.foreachRDD(lambda rdd: f(rdd))


In [ ]:
ssc = StreamingContext.getOrCreate('/tmp/checkpoint_v01',lambda: createContext())
ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()